In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Glucocorticoid_Sensitivity"
cohort = "GSE15820"

# Input paths
in_trait_dir = "../../input/GEO/Glucocorticoid_Sensitivity"
in_cohort_dir = "../../input/GEO/Glucocorticoid_Sensitivity/GSE15820"

# Output paths
out_data_file = "../../output/preprocess/Glucocorticoid_Sensitivity/GSE15820.csv"
out_gene_data_file = "../../output/preprocess/Glucocorticoid_Sensitivity/gene_data/GSE15820.csv"
out_clinical_data_file = "../../output/preprocess/Glucocorticoid_Sensitivity/clinical_data/GSE15820.csv"
json_path = "../../output/preprocess/Glucocorticoid_Sensitivity/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"ZBTB16, a glucocorticoid response gene in acute lymphoblastic leukemia, interferes with glucocorticoid-induced apoptosis"
!Series_summary	"Glucocorticoids (GCs) cause apoptosis in lymphoid lineage cells and are therefore widely used in the therapy of lymphoid malignancies. The molecular mechanisms of the anti-leukemic GC effects are, however, poorly understood. We have previously defined a list of GC-regulated candidate genes by Affymetrix-based whole genome comparative expression profiling in children with acute lymphoblastic leukemia (ALL) during systemic GC monotherapy and in experimental systems of GC-induced apoptosis. ZBTB16, a Zink finger and BOZ-domain containing transcriptional repressor, was one of the most promising candidates derived from this screen. To investigate its possible role in GC-induced apoptosis and cell cycle arrest, we performed conditional over-expression experiments in CCRF-CEM childhood ALL cells. Transgenic ZBTB16 alo

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, this appears to be gene expression data
# from Affymetrix Exon 1.0 microarrays studying ZBTB16 and glucocorticoid response
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Looking at the sample characteristics dictionary
# 2.1 Data Availability

# For trait (Glucocorticoid Sensitivity):
# The "treatment" in row 1 indicates treatment conditions including "Dex" (dexamethasone, a glucocorticoid)
# We can use this to determine glucocorticoid sensitivity
trait_row = 1

# For age: Not available in the sample characteristics
age_row = None

# For gender: Not available - these appear to be cell lines, not individual patients
gender_row = None

# 2.2 Data Type Conversion Functions

def convert_trait(value: str) -> int:
    """Convert treatment information to glucocorticoid sensitivity binary indicator.
    
    The "Dex" and "DexDox" treatments indicate exposure to dexamethasone (a glucocorticoid)
    which can be used as our trait indicator.
    """
    if not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Determine treatment type
    if value == "Dex" or value == "DexDox":
        return 1  # Exposed to glucocorticoid
    elif value == "none" or value == "Dox":
        return 0  # Not exposed to glucocorticoid
    else:
        return None

def convert_age(value: str) -> Optional[float]:
    """Convert age information to float."""
    # Age data is not available
    return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender information to binary (0=female, 1=male)."""
    # Gender data is not available
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Only execute if trait_row is not None
if trait_row is not None:
    # Assuming clinical_data is available from previous steps
    try:
        # Load clinical data if it exists
        clinical_data = pd.read_csv(os.path.join(in_cohort_dir, "clinical_data.csv"))
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the dataframe
        print("Preview of selected clinical features:")
        print(preview_df(selected_clinical_df))
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save to CSV
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
        
    except FileNotFoundError:
        print("Clinical data file not found. Skipping clinical feature extraction.")
    except Exception as e:
        print(f"Error during clinical feature extraction: {e}")


Clinical data file not found. Skipping clinical feature extraction.


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 69
Header line: "ID_REF"	"GSM397450"	"GSM397451"	"GSM397452"	"GSM397453"	"GSM397454"	"GSM397455"	"GSM397456"	"GSM397457"	"GSM397458"	"GSM397459"	"GSM397460"	"GSM397461"	"GSM397462"	"GSM397463"	"GSM397464"	"GSM397465"	"GSM397466"	"GSM397467"	"GSM397468"	"GSM397469"	"GSM397470"	"GSM397471"	"GSM397472"	"GSM397473"	"GSM397474"	"GSM397475"	"GSM397476"	"GSM397477"	"GSM397478"	"GSM397479"	"GSM397480"	"GSM397481"	"GSM397482"	"GSM397483"	"GSM397484"	"GSM397485"	"GSM397486"	"GSM397487"	"GSM397488"	"GSM397489"
First data line: "52_36nbg_gc10"	1.36755972312661	1.36703559676413	1.35410230935632	1.38059015791809	1.42740938904254	1.43124503160126	1.41567377618281	1.40124792419307	1.35540529604859	1.35799836513495	1.38758634012505	1.36094675480995	1.35763165856656	1.39344264875760	1.36455661100291	1.36552628300498	1.36612594197734	1.38366057436023	1.40126778404146	1.42285685315322	1.33998005284175	1.35451231570066	1.32936586717567	1.36449159029923	1.36183473620910	1.330197166

Index(['52_36nbg_gc10', '52_36nbg_gc11', '52_36nbg_gc12', '52_36nbg_gc13',
       '52_36nbg_gc14', '52_36nbg_gc15', '52_36nbg_gc16', '52_36nbg_gc17',
       '52_36nbg_gc18', '52_36nbg_gc19', '52_36nbg_gc20', '52_36nbg_gc21',
       '52_36nbg_gc22', '52_36nbg_gc23', '52_36nbg_gc24', '52_36nbg_gc25',
       '52_36nbg_gc3', '52_36nbg_gc4', '52_36nbg_gc5', '52_36nbg_gc6'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers in the dataset: '52_36nbg_gc10', '52_36nbg_gc11', etc.
# These are not standard human gene symbols, which are typically like BRCA1, TP53, etc.
# These appear to be probe IDs or custom identifiers that will need to be mapped to gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE15820
Line 6: !Series_title = ZBTB16, a glucocorticoid response gene in acute lymphoblastic leukemia, interferes with glucocorticoid-induced apoptosis
Line 7: !Series_geo_accession = GSE15820
Line 8: !Series_status = Public on May 07 2010
Line 9: !Series_submission_date = Apr 24 2009
Line 10: !Series_last_update_date = Jun 26 2012
Line 11: !Series_pubmed_id = 20435142
Line 12: !Series_summary = Glucocorticoids (GCs) cause apoptosis in lymphoid lineage cells and are therefore widely used in the therapy of lymphoid malignancies. The molecular mechanisms of the anti-leukemic GC effects are, however, poorly understood. We have previously defined a list of GC-regulated candidate genes by Affymetrix-based whol


Gene annotation preview:
{'ID': ['52_36nENST00000343609', '52_36nTRAN00000088867', '52_36nENST00000361444', '52_36nENST00000401567', '52_36nENST00000332556'], 'transcript_id': ['ENST00000343609', 'TRAN00000088867', 'ENST00000361444;ENST00000335791;TRAN00000088866', 'ENST00000401567', 'ENST00000332556'], 'gene_id': ['ENSG00000136840', 'ENSG00000136840', 'ENSG00000136840', 'ENSG00000218830', 'ENSG00000185896'], 'nr_probes': [45, 25, 49, 8, 50], 'probes': ['1000013;1821570;3542467;5826253;2703268;372673;939345;3829429;3465732;4704754;2926002;4384017;3619759;4707366;4988334;3744432;1489412;3923300;3488760;5449937;5498240;1104695;4952605;1066297;2631422;2795348;3778075;3701190;2556339;4473678;1271165;3013026;5982170;5971292;455529;1263516;1532691;2502837;6540389;6204608;3314611;3836325;3053558;2448962;4057529', '1000013;1821570;5492779;4126240;29898;866976;3542467;2703268;372673;939345;3465732;4704754;2926002;3619759;4707366;3923300;4952605;2631422;3701190;4473678;1271165;5982170;5971292;2

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the right columns for probe IDs and gene symbols from the gene annotation data
# Looking at the preview, we can see:
# - 'ID' column contains identifiers like '52_36nENST00000343609' which match the format of the gene expression data
# - 'symbol' column contains gene symbols like 'ST6GALNAC4', 'LAMP1'

# Let's extract these two columns for our mapping
prob_col = 'ID'
gene_col = 'symbol'

# First, let's check if the gene expression data's identifiers match the annotation IDs
gene_expression_ids = set(gene_data.index.tolist()[:5])  # Take a small sample
annotation_ids = set(gene_annotation[prob_col].tolist()[:5])

print(f"Sample gene expression IDs: {list(gene_expression_ids)[:5]}")
print(f"Sample annotation IDs: {list(annotation_ids)[:5]}")

# 2. Create the gene mapping dataframe
try:
    mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)
    print(f"Gene mapping created with {len(mapping_df)} rows")
    print("Sample mapping entries:")
    print(mapping_df.head())
except Exception as e:
    print(f"Error creating gene mapping: {e}")
    
    # Manual approach if library function fails
    mapping_df = gene_annotation[[prob_col, gene_col]].copy()
    mapping_df = mapping_df.dropna(subset=[gene_col])
    mapping_df = mapping_df.rename(columns={gene_col: 'Gene'})
    mapping_df = mapping_df.astype({prob_col: 'str'})
    print(f"Gene mapping created manually with {len(mapping_df)} rows")
    print("Sample mapping entries:")
    print(mapping_df.head())

# 3. Apply the gene mapping to convert probe expression to gene expression
try:
    # Check if our IDs in gene_data match those in the mapping
    common_ids = set(gene_data.index).intersection(set(mapping_df[prob_col]))
    print(f"Number of common IDs between expression data and mapping: {len(common_ids)}")
    
    # Apply the mapping to get gene expression data
    gene_data = apply_gene_mapping(gene_data, mapping_df)
    print(f"Gene expression data created with {gene_data.shape[0]} genes and {gene_data.shape[1]} samples")
    print("Sample gene expression data:")
    print(gene_data.head())
    
    # Save the gene expression data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")
    
except Exception as e:
    print(f"Error applying gene mapping: {e}")
    
    # Print more diagnostic information
    print("\nDiagnostic information:")
    print(f"gene_data index type: {type(gene_data.index[0])}")
    if 'ID' in mapping_df.columns:
        print(f"mapping_df ID column type: {type(mapping_df['ID'].iloc[0])}")


Sample gene expression IDs: ['52_36nbg_gc13', '52_36nbg_gc10', '52_36nbg_gc11', '52_36nbg_gc14', '52_36nbg_gc12']
Sample annotation IDs: ['52_36nENST00000401567', '52_36nENST00000361444', '52_36nENST00000332556', '52_36nTRAN00000088867', '52_36nENST00000343609']
Gene mapping created with 118098 rows
Sample mapping entries:
                      ID        Gene
0  52_36nENST00000343609  ST6GALNAC4
1  52_36nTRAN00000088867  ST6GALNAC4
2  52_36nENST00000361444  ST6GALNAC4
4  52_36nENST00000332556       LAMP1
5  52_36nTRAN00000107877       LAMP1
Number of common IDs between expression data and mapping: 118098


Gene expression data created with 18151 genes and 40 samples
Sample gene expression data:
       GSM397450  GSM397451  GSM397452  GSM397453  GSM397454  GSM397455  \
Gene                                                                      
A1BG    8.510088   8.778496   8.191638   9.090064   9.387474   9.093869   
A1CF   19.023268  20.432171  20.078757  21.372999  21.347969  23.153673   
A26A1   5.568398   5.678820   5.527850   5.642854   5.836115   5.679008   
A2LD1  13.905637  13.912146  12.940048  15.369641  16.542266  14.013720   
A2M    27.717106  27.142966  26.976324  28.835185  29.746781  31.796226   

       GSM397456  GSM397457  GSM397458  GSM397459  ...  GSM397480  GSM397481  \
Gene                                               ...                         
A1BG    9.241304   8.718551   9.075634   8.368584  ...   8.267192   8.349470   
A1CF   20.842212  21.226769  20.845009  19.277951  ...  21.011489  20.391923   
A26A1   5.793950   6.203565   5.674946   5.659406  ...   5.20945

Gene expression data saved to ../../output/preprocess/Glucocorticoid_Sensitivity/gene_data/GSE15820.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"Sample gene symbols after normalization: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

# 2. Since we didn't successfully save clinical data in previous steps, let's extract it again
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_df = get_background_and_clinical_data(matrix_file)

# Define conversion functions (from Step 2)
def convert_trait(value: str) -> int:
    """Convert treatment information to glucocorticoid sensitivity binary indicator."""
    if not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Determine treatment type
    if value == "Dex" or value == "DexDox":
        return 1  # Exposed to glucocorticoid
    elif value == "none" or value == "Dox":
        return 0  # Not exposed to glucocorticoid
    else:
        return None

def convert_age(value: str) -> Optional[float]:
    """Convert age information to float."""
    # Age data is not available
    return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender information to binary (0=female, 1=male)."""
    # Gender data is not available
    return None

# Extract clinical features with the identified row indices
trait_row = 1  # From Step 2
age_row = None  # From Step 2
gender_row = None  # From Step 2

# Extract clinical features
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_df,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print("Selected clinical features:")
print(selected_clinical_df)

# Save clinical data for future reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data columns preview:")
print(list(linked_data.columns[:10]))  # Show first 10 column names

# 4. Handle missing values
print("\nMissing values before handling:")
print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
if gene_cols:
    missing_genes_pct = linked_data[gene_cols].isna().mean()
    genes_with_high_missing = sum(missing_genes_pct > 0.2)
    print(f"  Genes with >20% missing: {genes_with_high_missing}")
    
    if len(linked_data) > 0:  # Ensure we have samples before checking
        missing_per_sample = linked_data[gene_cols].isna().mean(axis=1)
        samples_with_high_missing = sum(missing_per_sample > 0.05)
        print(f"  Samples with >5% missing genes: {samples_with_high_missing}")

# Handle missing values
cleaned_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {cleaned_data.shape}")

# 5. Evaluate bias in trait and demographic features
trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)

# 6. Final validation and save
note = "Dataset contains gene expression data from glucocorticoid sensitivity studies. "
if 'Age' in cleaned_data.columns:
    note += "Age data is available. "
if 'Gender' in cleaned_data.columns:
    note += "Gender data is available. "

is_gene_available = len(normalized_gene_data) > 0
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=True, 
    is_biased=trait_biased, 
    df=cleaned_data,
    note=note
)

# 7. Save if usable
if is_usable and len(cleaned_data) > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable or empty and was not saved")

Gene data shape after normalization: (18042, 40)
Sample gene symbols after normalization: ['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC']


Gene data saved to ../../output/preprocess/Glucocorticoid_Sensitivity/gene_data/GSE15820.csv


Selected clinical features:
                            GSM397450  GSM397451  GSM397452  GSM397453  \
Glucocorticoid_Sensitivity        0.0        0.0        0.0        0.0   

                            GSM397454  GSM397455  GSM397456  GSM397457  \
Glucocorticoid_Sensitivity        0.0        0.0        0.0        0.0   

                            GSM397458  GSM397459  ...  GSM397480  GSM397481  \
Glucocorticoid_Sensitivity        0.0        0.0  ...        1.0        1.0   

                            GSM397482  GSM397483  GSM397484  GSM397485  \
Glucocorticoid_Sensitivity        1.0        0.0        1.0        1.0   

                            GSM397486  GSM397487  GSM397488  GSM397489  
Glucocorticoid_Sensitivity        1.0        1.0        1.0        1.0  

[1 rows x 40 columns]
Clinical data saved to ../../output/preprocess/Glucocorticoid_Sensitivity/clinical_data/GSE15820.csv
Linked data shape: (40, 18043)
Linked data columns preview:
['Glucocorticoid_Sensitivity', 'A1BG

Data shape after handling missing values: (40, 18043)
For the feature 'Glucocorticoid_Sensitivity', the least common label is '1.0' with 15 occurrences. This represents 37.50% of the dataset.
The distribution of the feature 'Glucocorticoid_Sensitivity' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Glucocorticoid_Sensitivity/cohort_info.json


Linked data saved to ../../output/preprocess/Glucocorticoid_Sensitivity/GSE15820.csv
